In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os
from build_model import ActionRecognitionTrainer

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('../datasets/custom/dataset.csv')
df.head()

,clip_path,label
0,../datasets/custom/jogging\jogging.mp4,jogging
1,../datasets/custom/walking\walking.mp4,walking


In [3]:
df['label'].unique()

array(['jogging', 'walking'], dtype=object)

In [4]:
def three_class(x):
    if x not in ['golf', 'kick_ball', 'pushup', 'shoot_ball',  
             'shoot_bow', 'shoot_gun', 'swing_baseball', 
             'thow', 'BoxingPunchingBag', 'boxing', 'punch', 
             'kick', 'point', 'handwaving', 'wave']:
        return 0
    else:
        return 1

In [5]:
df['label'] = df['label'].apply(three_class)

In [6]:
classwise_df = []

for i in df['label'].unique():
    classwise_df.append(df[df['label']==i].reset_index())

In [7]:
num_classes = 1
trainer = ActionRecognitionTrainer(activation_fn='sigmoid', buffer_length=10, optimizer='adam', metrics=['accuracy'])
fusion_model = trainer.train_model(num_classes)


Building the RGB net...
Building the Pose net...
Building the fussion net...
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv3d_input (InputLayer)      [(None, 10, 640, 64  0           []                               
                                0, 3)]                                                            
                                                                                                  
 conv3d (Conv3D)                (None, 8, 638, 638,  5248        ['conv3d_input[0][0]']           
                                 64)                                                              
                                                                                                  
 max_pooling3d (MaxPooling3D)   (None, 8, 319, 319,  0           ['conv3d[0][0]']                 
                 

In [8]:
print(fusion_model.input_shape)

[(None, 10, 34), (None, 10, 640, 640, 3)]


In [9]:
print(fusion_model.output_shape)

(None, 1)


# Inference on Dataset

In [10]:
from ultralytics import YOLO
import cv2

In [11]:
model = YOLO('models/yolov8m-pose.pt') 

In [12]:
total_df = pd.DataFrame()
for df in classwise_df:
    total_df = pd.concat([total_df, df])

total_df = total_df.sample(len(total_df)).reset_index().drop(['level_0'], axis=1)

In [13]:
total_df.shape

(2, 3)

In [18]:
import cv2
import numpy as np
import pandas as pd
from random import shuffle
import os

def data_generator(total_df, batch_size=1, shuffle_data=True, resize=640):
    """
    Yields the next training batch.
    Suppose `total_df` is a pandas DataFrame with columns 'clip_path' and 'label'.
    """
    num_samples = len(total_df)
    samples = list(total_df.iterrows())  # Convert DataFrame to a list of (index, row) tuples

    while True:  # Loop forever so the generator never terminates
        if shuffle_data:
            shuffle(samples)

        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]

            # Initialise pose_buffer and labels arrays for this batch
            pose_buffer = []
            rbg_buffer = []
            labels = []

            # For each example
            for _, batch_sample in batch_samples:
                # Load video (X) and label (y)
                vid_path = batch_sample['clip_path']
                label = batch_sample['label']
                cap = cv2.VideoCapture(vid_path)
                frame_buffer = []
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        cap.release()
                        break

                    frame = cv2.resize(frame, (resize, resize))
                    if len(frame_buffer) < 10:
                        frame_buffer.append(frame)
                    elif len(frame_buffer) == 10:
                        batch = vp.process_video(frame_buffer)
                        batch = np.array(batch)[:, 0, :].reshape(-1, 10, 34)
                        pose_buffer.append(batch[0])
                        rbg_buffer.append(frame)
                        frame_buffer = []
                    else:
                        frame_buffer = []

                labels.extend([label] * len(pose_buffer))  # Extend the labels list for this video's frames

            # Make sure they're numpy arrays (as opposed to lists)
            pose_buffer = np.array(pose_buffer)
            rbg_buffer = np.array(rbg_buffer)
            labels = np.array(labels)

            print(pose_buffer.shape)
            print(rbg_buffer.shape)
            print(labels.shape)

            # The generator-y part: yield the next training batch
            yield [pose_buffer, rbg_buffer], labels  # ([(None, 10, 34), (None, 10, 640, 640, 3)], (None,1))
# Example usage:
# Assuming you have defined vp and fusion_model elsewhere
batch_size = 1
num_epochs = 1
vp = VideoProcessor(model, batch_size, max_frames=100, img_sz=32, show_stream=False)

train_generator = data_generator(total_df, batch_size=batch_size)

# Now, you can use this generator to train your model
# print(np.array(next(train_generator)).shape)
fusion_model.fit(train_generator, steps_per_epoch=len(total_df) // batch_size, epochs=num_epochs)


(1018, 1, 10, 34)
(1018, 640, 640, 3)
(1018,)


ValueError: in user code:

    File "c:\Users\aniru\anaconda3\envs\yolov8\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\aniru\anaconda3\envs\yolov8\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\aniru\anaconda3\envs\yolov8\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\aniru\anaconda3\envs\yolov8\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\aniru\anaconda3\envs\yolov8\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\aniru\anaconda3\envs\yolov8\lib\site-packages\keras\engine\input_spec.py", line 250, in assert_

In [67]:
total_df = pd.DataFrame()
for df in classwise_df:
    total_df = pd.concat([total_df, df])

total_df = total_df.sample(2).reset_index().drop(['level_0'], axis=1)

In [73]:
batch_size = 10
vp = VideoProcessor(model, batch_size, max_frames=100, img_sz=512, show_stream=False)

keypoints_df = pd.DataFrame()
class_keypoints_df = pd.DataFrame()

# for train_df in classwise_df:
label = []

pose_buffer = []
rgb_buffer = []

# for vid_path, label in zip(total_df['clip_path'], total_df['label']):
# for vid_path, label in zip(classwise_df[0][:2]['clip_path'], classwise_df[0][:2]['label']) :
for vid_path, label in [[0,0]] :
    # print(vid_path)
    cap = cv2.VideoCapture(vid_path)
    frame_buffer = []
    pose_buffer = []
    rgb_buffer = []
    while True:
        ret, frame = cap.read()
        if not ret:
            cv2.destroyAllWindows()
            break

        frame.resize((640, 640, 3))
        # print(frame.shape)
        
        if cv2.waitKey(10) ==27:
            break
        cv2.imshow("frame", frame)
        
        if len(frame_buffer)<10:
            frame_buffer.append(frame)
            
        elif len(frame_buffer)==10:
            batch = vp.process_video(frame_buffer)
            # print(np.array(batch[:,0,:]).shape)
            batch = np.array(batch)[0,:,:].reshape(-1,batch_size , 34)
            print(batch,end='\n********************************\n')
            pose_buffer.append(batch[0]) # batch.shape = (1,10,34)
            rgb_buffer.append(frame_buffer)
            frame_buffer=[]
            # break

        else:
            frame_buffer=[]

    print(np.array(pose_buffer).shape)
    print(np.array(rgb_buffer).shape)
    print(np.array([label]*len(pose_buffer)).shape)
    # fusion_model.fit(x = [np.array(pose_buffer), np.array(rgb_buffer)], y = np.array([label]*len(pose_buffer)))
    fusion_model.fit(x = [np.array(pose_buffer)], y = np.array([label]*len(pose_buffer)))


                
    
    

[[[0.55173292 0.44426999 0.6003336  0.37206407 0.47465982 0.38366449 0.66192031 0.39464903 0.37560005 0.42417598 0.74635673 0.62433152 0.2091094  0.65434084 0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.        ]
  [0.55452323 0.44917016 0.60291653 0.3741473  0.47645683 0.38533151 0.66277194 0.39610357 0.37568643 0.42509089 0.74632015 0.62259159 0.21321311 0.64846807 0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.        ]
  [0.55432224 0.44774299 0.60367785 0.37433248 0.47628903 0.38498063 0.6647696  0.39833107 0.37607286 0.42583632 0.74651885 0.62173223 0.21712499 0.64378481 0.         0.         0.         0.         0.         0.         0.         0.         0.       

KeyboardInterrupt: 

In [26]:
pose_buffer[0][0]

array([0.67196331, 0.51452093, 0.72848344, 0.44471197, 0.60920515, 0.45595284, 0.81252975, 0.45757737, 0.53810687, 0.47903714, 0.88968382, 0.73315597, 0.3788687 , 0.70668087, 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ])

In [21]:
model.track(0, show=True, save=False, stream=False,
                imgsz=512, device='0', verbose=False,
                tracker="bytetrack.yaml", conf=0.5, iou=0.5)

1/1: 0... Success ✅ (inf frames of shape 640x480 at 30.00 FPS)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

WARNING ⚠️ Waiting for stream 0


KeyboardInterrupt: 